<a href="https://colab.research.google.com/github/Luisamfer/MBA_IA_Generativa/blob/main/Chatbot_norma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Comando para baixar o PDF

In [ ]:
!pip install langchain faiss-cpu sentence-transformers
!pip install langchain-community
!pip install langchain_huggingface
!pip install chromadb
!pip install langchain_openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/ColabNotebooks/IA_GEN')

In [4]:
!wget -O portaria19.html http://www.in.gov.br/web/dou/-/portaria-n-19-de-21-de-marco-de-2025-619527337

--2025-04-17 19:12:11--  http://www.in.gov.br/web/dou/-/portaria-n-19-de-21-de-marco-de-2025-619527337
Resolving www.in.gov.br (www.in.gov.br)... 177.15.137.168
Connecting to www.in.gov.br (www.in.gov.br)|177.15.137.168|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://www.in.gov.br/web/dou/-/portaria-n-19-de-21-de-marco-de-2025-619527337 [following]
--2025-04-17 19:12:12--  https://www.in.gov.br/web/dou/-/portaria-n-19-de-21-de-marco-de-2025-619527337
Connecting to www.in.gov.br (www.in.gov.br)|177.15.137.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘portaria19.html’

portaria19.html         [   <=>              ] 146.16K   224KB/s    in 0.7s    

2025-04-17 19:12:14 (224 KB/s) - ‘portaria19.html’ saved [149670]



In [5]:
from langchain_community.document_loaders import BSHTMLLoader

loader = BSHTMLLoader("portaria19.html")
dados = loader.load()
#print(data[0].page_content)

## Código para popular um Banco de Dados Vetorial

In [7]:
!pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 17.6 MB/s eta 0:00:00


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
textos = text_splitter.split_documents(dados)
embedding_engine = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_db = Chroma.from_documents(textos, embedding_engine, persist_directory="/content/drive/MyDrive/Colab Notebooks/IA_GEN")

## Código para montar um RAG

In [9]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
n_documentos = 5

def format_docs(documentos):
    return "\n\n".join(documento.page_content for documento in documentos)

llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model = "gpt-4o-mini")
prompt = hub.pull("rlm/rag-prompt")

rag = (
    {
        "question": RunnablePassthrough(),
        "context": vector_db.as_retriever(k = n_documentos)
                    | format_docs
    }
    | prompt
    | llm
    | StrOutputParser()
)


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Código para perguntar ao RAG

In [11]:
prompt = "Qual a documentação obrigatória para inscrição no CAF?"
rag.invoke(prompt)

'A documentação obrigatória para a inscrição no Cadastro Nacional da Agricultura Familiar (CAF) inclui a apresentação de informações necessárias pelo requerente, sem omissões ou falsificações. Além disso, deve ser anexada a Declaração de Veracidade após o cadastramento. É fundamental que os documentos comprovem os requisitos legais ao enquadramento.'